<a href="https://colab.research.google.com/github/movie112/INU-DILAB/blob/main/NLP_pytorch/NLP_ch5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 파이토치로 배우는 자연어 처리: ch5
## 5. 단어와 타입 임베딩
5.1 임베딩을 배우는 이유    
5.2 예제: CBOW 임베딩 학습하기   
5.3 예제: 문서 분류에 사전 훈련된 임베딩을 사용한 전이 학습 

---


- NLP 작업을 구현할 때는 여러 종류의 이산적인 타입을 다뤄야 함.   
  - 예) __단어__(유한한 집합(어휘 사전)), 문자, 품사태깅, 개체명, 개체명타입, 파싱피처, 제품카탈로그의 아이템   
- `이산타입`: 기본적으로 유한한 집합에서 얻은 모든 입력 특성
- 이산타입을 밀집 벡터로 표현하는 것이 NLP에서 딥러닝 서공의 핵심
- `표현학습`, `임베딩` 이라는 용어는 이산 타입과 벡터 공간의 포인트 사이에 매핑을 학습하는 것
- `단어 임베딩(word embedding)`: 이산타입이 단어일 때 밀집 벡터 표현
  - 이 장에서는 __학습 기반__ 또는 __ 예측 기반__의 임베딩 방법을 알아보겠다.
  - 모델이 만들에징 후에 추가되는 것이 아니라 모델 자체의 기본 요소

---



## 5.1 임베딩을 배우는 이유
- 차원은 줄이면 계산을 효율적으로 수행
- 타운트 기반 표현은 여러 차원에 비슷한 정보를 중복해 인코딩한 고차원 벡터를 만든다.
  - 이런 벡터는 통계적 장점을 공유하지 못한다.
- 매우 고차원 입력은 머신러닝 최적화에서 실제로 문제가 될 수 있다.
  - 특이값분해, 주성분분석 과 같은 차원축소방법을 전통적으로 사용
  - 차원이 수백만 개일 때는 잘 적용되지 않음
- 작업에 특화된 데이터에서 학습된 표현은 현재 작업에 최적
  - TF-IDF 같이 경험적이거나 SVD 같이 저차원 장법은 임베딩의 최적화 목적이 해장 작업과 관련이 있는지 명확하지 않음
  

#### 5.1.1 임베딩의 효율성

<img src="https://gblobscdn.gitbook.com/assets%2F-LFzjkZt6ljMBd4YGVCn%2F-LX4NCwvtTXyg_vzXhF0%2F-LX4NGWF5GFb-sBQgMQ3%2F06-03-01.png?alt=media" height="200px" width="350px"></img>

- one-hot vector를 가중치 행렬의 모든 값에 곱한 후 각 행의 합을 계산하므로 비용 많이 들고 비효율적
- 정의에 따라 one-hot vector를 입력으로 받는 Linear 층의 가중치 행렬에는 one-hot vector의 크기와 같은 개수의 열이 있어야 함
- 행렬 곱셈을 수행할 때 결과 벡터는 실제로 0이 아닌 원소가 가르키는 행을 선택하여 만들어진다.
- 행렬 곱셈 단계를 건너 뛰고 직접 정수를 인덱스로 사용하여 선태된 행을 추출할 수 있다.

> - 실제로 임베딩은 one-hot vector 또는 카운트 기반 표현보다 낮은 차원 공간에서 단어를 표현하는 데 자주 사용

#### 5.1.2 단어 임베딩 학습 방법
- 임베딩 방법은 모두 단어만으로(레이블이 없는 데이터로) 학습되지만 지도학습방식을 사용
  - 이를 위해 데이터가 암묵적으로 레이블되어 있는 __보조 작업__을 구성
    - 단어 시퀀스가 주어지면 다음 단어 예측: `언어 모델링 작업`
    - 앞과 뒤의 단어 시퀀스가 주어지면 누락된 단어 예측
    - 단어 주어지면 위치에 관계없이 window 안에 등장할 단어 예측
 

#### 5.1.3 사전 훈련된 단어 임베딩


##### 임베딩 로드
- `Word2Vec`: 여러 임베딩 방법의 모음
- 임베딩은 다음과 같은 포맷으로 제공됩니다.
  - 각 줄은 임베딩되는 단어/타입으로 시작하고 그 뒤에 숫자 시퀀스(즉, 벡터 표현)
  - 이 시퀀스의 길이는 표현의 차원(`임베딩 차원`)
    - `임베딩 차원`은 보통 수백 개 정도
    - `토큰 타입`의 개수는 어휘 사전의 크기이며 백만 개 정도

In [ ]:
# annoy 패키지를 설치합니다.
!pip install annoy

     |████████████████████████████████| 646 kB 4.2 MB/s 
  Created wheel for annoy: filename=annoy-1.17.0-cp37-cp37m-linux_x86_64.whl size=391684 sha256=ad64db028d62279394b34cd8ed3ef4b15bbab94520ade69182680758846c4613
  Stored in directory: /root/.cache/pip/wheels/4f/e8/1e/7cc9ebbfa87a3b9f8ba79408d4d31831d67eea918b679a4c07
Successfully built annoy


- 임베딩을 효율적으로 로드하고 처리하는 `PreTrainedEmbeddings` 유틸리키 클래스
  - 빠른 조회를 위해 메모리 내에 모든 단어 벡터의 인덱스를 구축하고 근사 최근접 이웃 알고리즘을 구현한 `annoy` 패키지를 사용해 최근접 이웃 쿼리 수핸

In [ ]:
import torch
import torch.nn as nn
from tqdm import tqdm
from annoy import AnnoyIndex
import numpy as np

In [ ]:
class PreTrainedEmbeddings(object):
    """ 사전 훈련된 단어 벡터 사용을 위한 래퍼 클래스 """
    def __init__(self, word_to_index, word_vectors):
        """
        매개변수:
            word_to_index (dict): 단어에서 정수로 매핑
            word_vectors (numpy 배열의 리스트)
        """
        self.word_to_index = word_to_index
        self.word_vectors = word_vectors
        self.index_to_word = {v: k for k, v in self.word_to_index.items()}

        self.index = AnnoyIndex(len(word_vectors[0]), metric='euclidean')
        print("인덱스 만드는 중!")
        for _, i in self.word_to_index.items():
            self.index.add_item(i, self.word_vectors[i])
        self.index.build(50)
        print("완료!")
        
    @classmethod
    def from_embeddings_file(cls, embedding_file):
        """사전 훈련된 벡터 파일에서 객체를 만듭니다.
        
        벡터 파일은 다음과 같은 포맷입니다:
            word0 x0_0 x0_1 x0_2 x0_3 ... x0_N
            word1 x1_0 x1_1 x1_2 x1_3 ... x1_N
        
        매개변수:
            embedding_file (str): 파일 위치
        반환값:
            PretrainedEmbeddings의 인스턴스
        """
        word_to_index = {}
        word_vectors = []

        with open(embedding_file) as fp:
            for line in fp.readlines():
                line = line.split(" ")
                word = line[0]
                vec = np.array([float(x) for x in line[1:]])
                
                word_to_index[word] = len(word_to_index)
                word_vectors.append(vec)
                
        return cls(word_to_index, word_vectors)

embeddings = \
  PreTrainedEmbeddings.from_embeddings_file('glove.6b.100d.txt')

##### 단어 임베딩 사이의 관계
- 단어 임베딩의 핵십 기능은 단어 사용에서 규칙적으로 나타나는 구문과 의니 관계를 인코딩하는 것
  - ex. 고양이와 개의 임베딩은 오리/코끼리 같은 동물의 임베딩보다 훨씬 가깝
- word1 : word2 :: word3 : _____
  - 유추

---


## 5.2 예제: CBOW 임베딩 학습하기
- CBOW 모델은 다중 분류 작업
  - 단어 텍스트를 스캔하여 단어의 문맥 window를 만든 후 문맥 window에서 중앙의 단어를 제거하고 문맥 window를 사용해 누락된 단어를 예측
  - 직관적으로 이를 빈칸 채우기 작업처럼 생각할 수 있습니다.
  - 모델은 단어가 누락된 문장에서 누락 단어가 무엇인지 파악하는 역할

---

- 예제에서는 임베딩 행렬을 캡슐화하는 파이토치 모듈인 nn.Embedding 층을 소개
  - Embedding 층을 사용해 코튼의 정수 ID를 신경망 계산에 사용되는 벡터로 매핑
  - optimizer는 모델 가중치를 업데이트할 때 이 벡터값도 업데이트해서 손실을 최소화
  - 모델은 이 과정에서 해당 작업에 가장 유용한 방식으로 단어 임베딩하는 밥법을 배웁니다.
- 예제의 나머지 부분
  - 프랑켄슈타인 데이터셋 소개
  - 토큰에서 벡터의 미니배치를 만드는 벡터화 파이프라인 설명
  - CBOW 분류 모델과 Embedding 층의 사용 방법을 간략하게 설명
  - 훈련 과정 다룸
  - 모델 평가, 추론 및 모델 검사 방법 설명


#### 5.2.1 프랑켄슈타인 데이터셋
- CBOW 작업: 왼쪽 문맥과 오른쪽 문맥을 사용해 단어를 예측합니다. 문맥 window 길이는 양쪽으로 2입니다. 텍스트 위를 슬라이딩하는 window가 지도 학습 샘플을 생성합니다. 
  - 각 샘플의 타깃 단어는 가운데 단어 
  - 길이가 2 가 아닌 window는 적절하게 패딩됨

---

- 데이터셋을 구성하는 마지막 단계는 데이터를 훈련, 검증, 테스트 세트로 분할하는 것
- 훈련 및 검증 세트는 모델 훈련 중에 사용
  - 훈련 세트는 파라미터를 업데이트하는 데 사용하고 검증 세트는 모델의 성능을 측정하는 데 사용
- 테스트 세트는 딱 한 번만 사용해 측정의 편향을 줄임
- 만들어진 윈도 데이터셋과 타깃은 판다스 데이터프레임으로 로드되고 CBOWDataset 클래스에서 인덱싱

> - 사용되는 정확한 window 크기는 하이퍼파라미터이며 CBOW에서 매우 중요
    - 윈도가 너무 크면 모델이 규칙성을 감지하지 못할 수 있고 너무 작으면 흥미로운 의존성을 놓칠 수 있음.





#### 5.2.2 Vocabulary, Vectorizer. DataLoader
- 텍스트를 벡터의 미니배채로 변환하는 파이프라인은 기존 구현과 대부분 같다.
- Vectorizer: one-hot vector을 만들기 대신 문맥의 인덱스를 나타내는 정수 벡터를 만들어 반환
  - 문맥의 토큰 수가 최대 길이보다 작으면 나머지는 0으로 채움

In [ ]:
class CBOWVectorizer(object):
    """ 어휘 사전을 생성하고 관리합니다 """
    def __init__(self, cbow_vocab):
        """
        매개변수:
            cbow_vocab (Vocabulary): 단어를 정수에 매핑합니다
        """
        self.cbow_vocab = cbow_vocab

    def vectorize(self, context, vector_length=-1):
        """
        매개변수:
            context (str): 공백으로 나누어진 단어 문자열
            vector_length (int): 인덱스 벡터의 길이 매개변수
        """

        indices = [self.cbow_vocab.lookup_token(token) for token in context.split(' ')]
        if vector_length < 0:
            vector_length = len(indices)

        out_vector = np.zeros(vector_length, dtype=np.int64)
        out_vector[:len(indices)] = indices
        out_vector[len(indices):] = self.cbow_vocab.mask_index

        return out_vector
    
    @classmethod
    def from_dataframe(cls, cbow_df):
        """데이터셋 데이터프레임에서 Vectorizer 객체를 만듭니다
        
        매개변수::
            cbow_df (pandas.DataFrame): 타깃 데이터셋
        반환값:
            CBOWVectorizer 객체
        """
        cbow_vocab = Vocabulary()
        for index, row in cbow_df.iterrows():
            for token in row.context.split(' '):
                cbow_vocab.add_token(token)
            cbow_vocab.add_token(row.target)
            
        return cls(cbow_vocab)

    @classmethod
    def from_serializable(cls, contents):
        cbow_vocab = \
            Vocabulary.from_serializable(contents['cbow_vocab'])
        return cls(cbow_vocab=cbow_vocab)

    def to_serializable(self):
        return {'cbow_vocab': self.cbow_vocab.to_serializable()}

#### 5.2.3 CBOWClassifier 모델
- CBOWClassifier의 핵심 3단계
  - Embedding 층을 사용해 문맥의 단어를 나타내는 인덱스를 각 단어에 대한 벡터로 만듦
  - 전반적인 문맥을 감지하도록 벡터 결합
  - Linear 층에서 문맥 벡터를 사용해 예측 벡터 계산
    - 이 예측 벡터는 전체 어휘 사전에 대한 확률 분포
    - 예측 벡터에서 가장 큰(가능성 높은) 값이 타깃 단어(문맥에서 누락된 가운데 단어)에 대한 예측을 나타냄
  - 여기의 Embedding 층은 하이퍼파라미터 2개로 제어
    - 임베딩 개수(어휘 사전의 크기)
    - 임베딩 크기(임베딩 차원_
    - 매개변수: padding_idx: 데이터 포인트 길이가 모두 같지 않을 때 Embedding 층에 패딩하는데 사용
    - 이 층은 해당 인덱스에 상응하는 벡터와 gradient를 모두 0으로 만듦

In [ ]:
class CBOWClassifier(nn.Module): # Simplified cbow Model
    def __init__(self, vocabulary_size, embedding_size, padding_idx=0):
        """
        매개변수:
            vocabulary_size (int): 어휘 사전 크기, 임베딩 개수와 예측 벡터 크기를 결정합니다
            embedding_size (int): 임베딩 크기
            padding_idx (int): 기본값 0; 임베딩은 이 인덱스를 사용하지 않습니다
        """
        super(CBOWClassifier, self).__init__()
        
        self.embedding =  nn.Embedding(num_embeddings=vocabulary_size, 
                                       embedding_dim=embedding_size,
                                       padding_idx=padding_idx)
        self.fc1 = nn.Linear(in_features=embedding_size,
                             out_features=vocabulary_size)

    def forward(self, x_in, apply_softmax=False):
        """분류기의 정방향 계산
        
        매개변수:
            x_in (torch.Tensor): 입력 데이터 텐서 
                x_in.shape는 (batch, input_dim)입니다.
            apply_softmax (bool): 소프트맥스 활성화 함수를 위한 플래그
                크로스-엔트로피 손실을 사용하려면 False로 지정합니다
        반환값:
            결과 텐서. tensor.shape은 (batch, output_dim)입니다.
        """
        x_embedded_sum = F.dropout(self.embedding(x_in).sum(dim=1), 0.3)
        y_out = self.fc1(x_embedded_sum)
        
        if apply_softmax:
            y_out = F.softmax(y_out, dim=1)
            
        return y_out